This notebook can be run on the Copernicus Dataspace Jupyterhub but running the following package installation cell first

In [1]:
!pip install eomaji@git+https://github.com/DHI/EOMAJI-OpenEO-toolbox.git

  Cloning https://github.com/DHI/EOMAJI-OpenEO-toolbox.git to /tmp/pip-install-80e33qj2/eomaji_8555de1f92294ad3b88e51296b7aec39
  Running command git clone --filter=blob:none --quiet https://github.com/DHI/EOMAJI-OpenEO-toolbox.git /tmp/pip-install-80e33qj2/eomaji_8555de1f92294ad3b88e51296b7aec39
  fatal: could not read Username for 'https://github.com': terminal prompts disabled
  fatal: could not read Username for 'https://github.com': terminal prompts disabled
Username for 'https://github.com': ^C
ERROR: Operation cancelled by user


# Data Mining Sharpener Workflow

In [1]:
import openeo
from eomaji.workflows.decision_tree_sharpener import run_decision_tree_sharpener
from eomaji.utils import draw_aoi

### Set up the OpenEO connection

In [10]:
connection = openeo.connect("https://openeo.dataspace.copernicus.eu")
connection.authenticate_oidc()

Authenticated using refresh token.


<Connection to 'https://openeo.dataspace.copernicus.eu/openeo/1.2/' with OidcBearerAuth>

### Define the area of interest
You can either insert a polygon:

In [3]:
aoi = {
    "type": "Polygon",
    "coordinates": [[[8.1733155,56.554395],[8.1733155,56.554395],[8.1671337,56.4854867],[8.2378811,56.5051875],[8.2323862,56.5464504],[8.1733155,56.554395]]]
}

Or use the map to draw the area of interest:

In [4]:
m, polygons = draw_aoi()
m

Map(center=[40, 10], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

In [14]:
aoi = polygons[-1]

In [15]:
aoi

{'type': 'Polygon',
 'coordinates': [[[12.267724, 57.381261],
   [12.263604, 57.275247],
   [12.341877, 57.296771],
   [12.267724, 57.381261]]]}

### Select the date, the collections you want to use and load the datacubes
The low resolution mask is optional

In [16]:
date = "2023-06-11"
high_resolution_collection = "SENTINEL2_L2A"
low_resolution_collection = "SENTINEL3_SLSTR_L2_LST"
low_resolution_band = ["LST"]
low_resolution_mask_band = ["confidence_in"]

In [17]:
# Load high-resolution collection
high_resolution_cube = connection.load_collection(
    high_resolution_collection,
    spatial_extent=aoi,
    temporal_extent=date,
)

# Load low-resolution collection
low_resolution_cube = connection.load_collection(
    low_resolution_collection,
    spatial_extent=aoi,
    temporal_extent=date,
    bands=low_resolution_band,
    properties={"timeliness": lambda x: x=="NT",
            "orbitDirection": lambda x: x=="DESCENDING"}
)

# Cloud mask layer
mask_cube = connection.load_collection(
    low_resolution_collection,
    temporal_extent=date,
    spatial_extent=aoi,
    bands=low_resolution_mask_band,
    properties={"timeliness": lambda x: x=="NT",
                "orbitDirection": lambda x: x=="DESCENDING"}
)
mask_cube_binary = mask_cube.apply(lambda x: (x < 16384)*1.0)

### Run Data Mining Sharpener

In [18]:
sharpened_data = run_decision_tree_sharpener(
    high_resolution_cube = high_resolution_cube,
    low_resolution_cube = low_resolution_cube,
    low_resolution_mask = mask_cube_binary,
    mask_values = [1],
    cv_homogeneity_threshold = 0,
    moving_window_size = 30,
    disaggregating_temperature = True,
    n_jobs = 3,
    n_estimators = 30,
    max_samples = 0.8,
    max_features = 0.8,
)

/usr/lib/python3/dist-packages/osgeo/gdal.py:315: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
Warning 1: /tmp/tmplrac4gve.tiff: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.


Homogeneity CV threshold: 0.25
Number of training elements for is 117 representing 100% of avaiable low-resolution data.
Homogeneity CV threshold: 0.25
Number of training elements for is 117 representing 100% of avaiable low-resolution data.


Warning 1: /tmp/tmplrac4gve.tiff: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.


0
Saved MEM
Saved MEM
Saved MEM
Saved MEM
Saved MEM
Saved MEM


In [19]:
sharpened_data

<xarray.Dataset> Size: 3MB
Dimensions:    (band: 1, y: 1199, x: 519)
Coordinates:
  * x          (x) float64 4kB 3.35e+05 3.35e+05 ... 3.402e+05 3.402e+05
  * y          (y) float64 10kB 6.363e+06 6.363e+06 ... 6.351e+06 6.351e+06
  * band       (band) int64 8B 1
Data variables:
    band_data  (band, y, x) float32 2MB nan nan nan nan nan ... nan nan nan nan
Attributes:
    crs:        PROJCS["WGS 84 / UTM zone 33N",GEOGCS["WGS 84",DATUM["WGS_198...
    transform:  (335020.0, 10.0, 0.0, 6363140.0, 0.0, -10.0)

### Save data or continue the aggregations

In [11]:
sharpened_data.band_data.rio.to_raster('output.tif')